This notebook contains evaluation of BigBirdPegasus Model trained on arxiv research paper dataset. ROUGE is used for measuring accuracy. The test example used is from BigPatent dataset.
https://huggingface.co/docs/transformers/model_doc/bigbird_pegasus#transformers.BigBirdPegasusForConditionalGeneration

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.4 MB/s eta 0:00

In [ ]:
from datasets import load_dataset

#textiles/paper
ds = load_dataset("big_patent", "d")  # only 'a' CPC codes


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10164 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/565 [00:00<?, ? examples/s]

Dataset big_patent downloaded and prepared to /root/.cache/huggingface/datasets/big_patent/d/2.1.2/bc8ec8bdf469c0da5fef04becd32bb3b0b34df0b0baa088ae1237628dd7a9caa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#import pandas as pd
# put data into dataFrames and csv
#df_train = pd.DataFrame(ds['train'])
#df_val = pd.DataFrame(ds['validation'])
#df_test = pd.DataFrame(ds['test'])
#df_train.to_csv("train.csv",index=False)
#df_val.to_csv("val.csv",index=False)
#df_test.to_csv("test.csv",index=False)

In [ ]:
# cleaning text
import re

def remove_sentence_with_fig(paragraph):


    # first we remove . after FIG
    paragraph = re.sub(r"FIG.","FIG",paragraph)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    updated_paragraph = ' '.join(filtered_sentences)

    return updated_paragraph

def clean_text(text):

    # replace ° degree with
    text = re.sub(r"°", " degree", text)

    # replace US Patent
    text = re.sub(r" U.S. Pat. No. [0-9,]+", "Patent", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)


    text = text.strip()

    return text


text = ds['test'][0]["description"]
cleaned_text = clean_text(text)
text = remove_sentence_with_fig(cleaned_text)

print(cleaned_text)
print(text)

CROSS-REFERENCE TO RELATED APPLICATIONS The present application claims priority from Australian Provisional Patent Application No 2006903417 filed on 26 Jun. 2006, the content of which is incorporated herein by reference. INTRODUCTION TO THE INVENTION This invention relates to the packaging of consumer products and relates particularly to products that are formed of pulp material, particularly pulp material formed of waste paper, waste timber, waste fabric material, virgin timber and other similar pulp material. The invention more particularly relates to high quality products carrying high definition printing or other decoration. BACKGROUND TO THE INVENTION The creation of compelling and high quality packaging for consumer durables is well established and is executed in a variety of forms and formats known in the prior art but with each of the prior art, the formats and methodologies having their own particular limitations. The consumer market demands increasing colour, vibrancy and no

In [ ]:
# divide the text into paragraphs of 3500
text1 = text.split()
text_lst = []
for i in range(0,len(text1),3500):
  tmp = " ".join(text1[i:i+3500])
  text_lst.append(tmp)


In [ ]:
# exploring the dataset
print(ds['test'][0]["description"])

print("\n\n\n-------------")
print(ds['test'][0]["abstract"])

CROSS-REFERENCE TO RELATED APPLICATIONS 
     The present application claims priority from Australian Provisional Patent Application No 2006903417 filed on 26 Jun. 2006, the content of which is incorporated herein by reference. 
     INTRODUCTION TO THE INVENTION 
     This invention relates to the packaging of consumer products and relates particularly to products that are formed of pulp material, particularly pulp material formed of waste paper, waste timber, waste fabric material, virgin timber and other similar pulp material. The invention more particularly relates to high quality products carrying high definition printing or other decoration. 
     BACKGROUND TO THE INVENTION 
     The creation of compelling and high quality packaging for consumer durables is well established and is executed in a variety of forms and formats known in the prior art but with each of the prior art, the formats and methodologies having their own particular limitations. The consumer market demands incr

In [ ]:
# For cleaning predicted text
def clean_predicted_text(text):


    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)

    #replace new line character
    text = re.sub(r"<n>", "\n", text)

    #replace start of paragraph
    text = re.sub(r"<s>", "", text)

    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)




    return text

In [ ]:
#from transformers import AutoTokenizer, BigBirdModel
from transformers import AutoTokenizer, BigBirdPegasusForConditionalGeneration
import torch

# summary expected length
#expected_length = len(ds['test'][0]["abstract"])//2
# by default its in `block_sparse` mode with num_random_blocks=3, block_size=64
#model = BigBirdModel.from_pretrained("google/bigbird-roberta-large")
#tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-large")
# you can change `attention_type` to full attention like this:
#model = BigBirdModel.from_pretrained("google/bigbird-roberta-large")
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

# you can change `block_size` & `num_random_blocks` like this:
#model = BigBirdModel.from_pretrained("google/bigbird-roberta-large", block_size=16, num_random_blocks=2)
predictions = []
#pred_summary_length = expected_length//len(text_lst)
for i in range(0,len(text_lst)):
  tmp = text_lst[i]
  encoded_input = tokenizer([tmp], return_tensors='pt', max_length=4096, truncation=True)
  #output1 = model(**encoded_input1)
  #prediction1 = model.generate(**encoded_input1)
  prediction = model.generate(encoded_input["input_ids"], num_beams=4, max_length=100)
  prediction = clean_predicted_text(tokenizer.batch_decode(prediction,skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
  predictions.append(prediction)



In [ ]:
from rouge_score import rouge_scorer
abstract = ds['test'][0]["abstract"]
predicted_abstract = " ".join(predictions)
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(abstract, predicted_abstract)

In [ ]:
predicted_abstract

'in this paper , we describe a new kind of packaging which we call a hybrid packaging , in which a part of the packaging is made out of recycled material while the other part is made out of new material . we show that such a hybrid packaging can be made up of two types of materials , one of which is a conventional paperboard and the other is of a non - conventional paperboard . we show that such a hybrid packaging can be this article describes an automated mould mould printing process in which the final product is an embodiment of the desired shape and appearance of the final mould mould mould , the printing process is selected from a pre-form and the final product is an embodiment of the desired shape and appearance of the final mould mould mould , the printing is selected from a pre-form and the final product is an embodiment of the desired appearance of the final mould mould mould , the printing is selected from a pre-form'

In [ ]:
scores
#rouge score with 100 max_length tokens

{'rouge1': Score(precision=0.3, recall=0.4811320754716981, fmeasure=0.3695652173913043),
 'rouge2': Score(precision=0.05917159763313609, recall=0.09523809523809523, fmeasure=0.072992700729927),
 'rougeL': Score(precision=0.17647058823529413, recall=0.2830188679245283, fmeasure=0.2173913043478261)}